In [ ]:
import re
import difflib


In [23]:
def extract_functions(file_content):
    # 正则表达式模式以匹配C/C++函数声明
    function_pattern = r'(\w+)\s+(\w+)\s*\(([^)]*)\)\s*{'
    functions = re.findall(function_pattern, file_content)
    return functions

In [24]:
def find_function_body(return_type, function_name, functions_lines):
    key = return_type + " " + function_name
    start_line = [index for index, item in enumerate(functions_lines, 0) if key in item][0]
    in_function_body = False
    brace_count = 0  # 用于跟踪花括号的嵌套层级
    for i, line in enumerate(functions_lines[start_line:]):
        if not in_function_body:
            if '{' in line:
                in_function_body = True
                brace_count = 1
        else:
            if '{' in line:
                brace_count = brace_count + 1
            if "}" in line:
                brace_count = brace_count - 1
            if brace_count == 0:
                if start_line + i < len(functions_lines):
                    return functions_lines[start_line:start_line + i + 1]
                else:
                    return functions_lines[start_line:-1]

In [25]:
def find_called_functions(function_body):
    # 正则表达式模式以匹配函数调用
    call_pattern = r'\b(\w+)\s*\('
    called_functions = re.findall(call_pattern, function_body)
    return called_functions


In [26]:
with open('foo.c', 'r') as file:
    file_content = file.read()

cleaned_code = re.sub(r'/\*.*?\*/', '', file_content, flags=re.DOTALL)  # 使用正则表达式匹配和删除多行注释
cleaned_code = re.sub(r'//.*', '', cleaned_code)  # 使用正则表达式匹配和删除单行注释

functions_lines = [item for item in cleaned_code.split('\n') if item]
functions = extract_functions(file_content)
function_dic = {}

In [27]:
for return_type, function_name, function_para in functions:
    print("\n")
    print("return value" + return_type)
    print("fuction name" + function_name)
    print("paraments" + function_para)
    function_body = find_function_body(return_type, function_name, functions_lines)

    for line in function_body[1:]:
        if line != '{':
            print("first line" + line)
            break;
    for line in reversed(function_body):
        if line != '}':
            print("last line" + line)
            break;

    function_list = []
    for line in function_body[1:]:
        called_functions = find_called_functions(line)
        if len(called_functions) != 0:
            function_list = function_list + called_functions

    function_list = list(set(function_list))
    print("used function: " + str(function_list))

    function_dic[function_name]=function_body





返回值void
函数名pow_related_errors_001
参数
起始行	double num=10^2;
末尾行        dsink = ans;
调用函数: ['pow']


返回值void
函数名pow_related_errors_002
参数
起始行	double num=1.004;
末尾行        dsink = ans;
调用函数: ['pow']


返回值void
函数名pow_related_errors_003
参数
起始行	double arr[]={2.0,1.2,3.9,10^3,4.0};
末尾行        dsink = ans;
调用函数: ['for', 'sizeof', 'pow']


返回值void
函数名pow_related_errors_004
参数
起始行	double arr[]={2.0,1.2,3.9,10^3,4.0};
末尾行        dsink = ans;
调用函数: ['for', 'sizeof', 'pow']


返回值void
函数名pow_related_errors_005
参数
起始行	double arr[]={2.0,1.2,3.9,10^3,4.0};
末尾行        dsink = ans;
调用函数: ['for', 'sizeof', 'pow']


返回值double
函数名pow_related_errors_006_func_001
参数
起始行	return 10^3;
末尾行	return 10^3;
调用函数: []


返回值void
函数名pow_related_errors_006
参数
起始行	double exponent=2.0;	
末尾行        dsink = ans;
调用函数: ['pow', 'pow_related_errors_006_func_001']


返回值void
函数名pow_related_errors_007
参数
起始行	double exponent=2;
末尾行        dsink = ans + ans1;
调用函数: ['sizeof', 'malloc', 'pow', 'free']


返回值void
函数名pow_related_errors_

In [28]:
def calculate_diff(function1, function2):
    # 使用difflib计算两个函数之间的差异
    f1=str(function_dic[function1])
    f2=str(function_dic[function2])
    differ = difflib.Differ()
    diff = differ.compare(f1, f2)
    return '\n'.join(diff)


keys_list = list(function_dic.keys())
differ=calculate_diff(keys_list[0],keys_list[1])
print(differ)


  [
  '
  v
  o
  i
  d
   
  p
  o
  w
  _
  r
  e
  l
  a
  t
  e
  d
  _
  e
  r
  r
  o
  r
  s
  _
  0
  0
- 1
+ 2
  (
  )
  '
  ,
   
  '
  {
  '
  ,
   
  '
  \
  t
  d
  o
  u
  b
  l
  e
   
  n
  u
  m
  =
  1
+ .
  0
- ^
- 2
+ 0
+ 4
  ;
  '
  ,
   
  '
  \
  t
  d
  o
  u
  b
  l
  e
   
  e
  x
  p
  o
  n
  e
  n
  t
  =
+ 3
- 1
- 0
- ^
- 2
  ;
  '
  ,
   
  '
  \
  t
  d
  o
  u
  b
  l
  e
   
  a
  n
  s
  ;
  '
  ,
   
  '
  \
  t
  a
  n
  s
  =
  p
  o
  w
  (
  n
  u
  m
  ,
  e
  x
  p
  o
  n
  e
  n
  t
  )
  ;
   
   
  '
  ,
   
  '
   
   
   
   
   
   
   
   
  d
  s
  i
  n
  k
   
  =
   
  a
  n
  s
  ;
  '
  ,
   
  '
  }
  '
  ]


I used Python and regular expressions to complete the task, but I don't think this is the best solution. After writing this code, I also searched for other methods, including using clang and pycparser to generate abstract syntax trees to achieve the same effect. However, I believe that the form of regular expressions is the best way for beginners, especially for query question 4, I think there may be some issues. I think it may be a practical issue with Jupyter Lab.